In [1]:
# import the implementation
%run -i implementation.ipynb

In [ ]:
preproc_pipline = preproc_sd

target_features = ['efs', 'efs_time']
methods_list = ['cph', 'h_cph', 'xgb_aft', 'h_xgb_aft', 'rf_xgb', 'rsf', 'h_rsf', 'cat_aft']
sc_indexes = -1.0 * np.ones((1, len(methods_list))) 
sc_indexes = pd.DataFrame(data = sc_indexes, columns = methods_list)

X_train = df_train.drop(columns = target_features)
y_train = df_train[target_features]

X_test = df_test.drop(columns = target_features)
y_test = df_test[target_features] 

preproc_pipline.fit(X_train)
X_train_preproc = preproc_pipline.transform(X_train)
X_test_preproc = preproc_pipline.transform(X_test)
preds_rf_xgb = rf_xgb(X_train, y_train, X_test, y_test)

preds_hcph = hybrid_cph_model(X_train_preproc, y_train, X_test_preproc)
preds_hxgb = hybrid_xgb_aft_model(X_train_preproc, y_train, X_test_preproc)
preds_hrsf = hybrid_rsf_model(X_train_preproc, y_train, X_test_preproc)

preds_cph = cph_model(X_train_preproc, y_train, X_test_preproc)
preds_xgb = xgb_aft_model(X_train_preproc, y_train, X_test_preproc, params = params)
preds_cb = cb_aft_model(X_train, y_train, X_test, y_test, cb_preproc_sd)
preds_rsf = rsf_model(X_train_preproc, y_train, X_test_preproc)


solution = df_test

score_rf_xgb, c_index_rf_xgb = eval(preds_rf_xgb, X_test, solution)
sc_indexes['rf_xgb'] = score_rf_xgb

score_cph, c_index_cph = eval(preds_cph, X_test, solution)
score_xgb, c_index_xgb = eval(preds_xgb, X_test, solution)
score_cb, c_index_cb = eval(preds_cb, X_test, solution)
score_rsf, c_index_rsf = eval(preds_rsf, X_test, solution)

score_hcph, c_index_hcph = eval(preds_hcph, X_test, solution)
score_hxgb, c_index_hxgb = eval(preds_hxgb, X_test, solution)
score_hrsf, c_index_hrsf = eval(preds_hrsf, X_test, solution)

sc_indexes['h_cph'] = score_hcph
sc_indexes['h_xgb_aft'] = score_hxgb
sc_indexes['h_rsf'] = score_hrsf

sc_indexes['cph'] = score_cph
sc_indexes['xgb_aft'] = score_xgb
sc_indexes['rsf'] = score_hrsf

sc_indexes['cat_aft'] = score_cb

output = pd.DataFrame(sc_indexes, columns = methods_list)
output.to_csv('test_results_extended.csv', sep= '\t', header= True)